## 1) Сбор корпуса
Для корпуса я собрала все стихотворения Блока, сохранила в словарь.

In [18]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [19]:
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alinazabolotskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
URL = "https://slova.org.ru/blok/"
HEADERS = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36", "accept" : "*/*"} 
HOST = 'https://slova.org.ru'
PATH = r'Documents\avtobreya_project.csv' 
ua = UserAgent(verify_ssl=False)
session = requests.session()
author = "Александр Блок"
pos_list = {'A',
 'ADV',
 'ADVPRO',
 'ANUM',
 'APRO',
 'CONJ',
 'INTJ',
 'NUM',
 'PART',
 'PR',
 'S',
 'SPRO',
 'V'}

In [6]:
def get_poem(url):
    html = requests.get(url, headers=HEADERS) 
    print(f"{html.status_code} -- {url}")
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, 'html.parser')
        data = []
        text = soup.find('div', class_='grid-col-3').find_next('pre').get_text()
        for em in soup.find('div', class_='grid-col-3').find_next('pre').find_all('em'):
            text = text.replace(em.get_text(), '')
    return text
        

In [7]:
def poem_walker(url):
    html = requests.get(url, headers=HEADERS) 
    print(f"{html.status_code} -- {url}")
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, 'html.parser')
        data = {}
        for item in soup.find('div', class_='grid-col-1').find_all('a'):
            name = item.get_text()
            link = item.get('href')
            poem = get_poem(HOST+link)
            data.update({name: poem})
   
    return data

In [8]:
poems = poem_walker(URL)

200 -- https://slova.org.ru/blok/
200 -- https://slova.org.ru/blok/aviator/
200 -- https://slova.org.ru/blok/iavidelmrak/
200 -- https://slova.org.ru/blok/antverpen/
200 -- https://slova.org.ru/blok/barka_zhizni_vstala/
200 -- https://slova.org.ru/blok/begutnevernye/
200 -- https://slova.org.ru/blok/bezveryvboga/
200 -- https://slova.org.ru/blok/bezmenjab/
200 -- https://slova.org.ru/blok/bezradostnyevskhodjat/
200 -- https://slova.org.ru/blok/beloynochiiu/
200 -- https://slova.org.ru/blok/belyjkonchut/
200 -- https://slova.org.ru/blok/blagoslovlyaiu_vse/
200 -- https://slova.org.ru/blok/blazhennoty/
200 -- https://slova.org.ru/blok/blesnulo_v_glazakh/
200 -- https://slova.org.ru/blok/bozhekakzhizn/
200 -- https://slova.org.ru/blok/bolotistim/
200 -- https://slova.org.ru/blok/bolotnyechertenjatki/
200 -- https://slova.org.ru/blok/bolotnyjpopik/
200 -- https://slova.org.ru/blok/bolotoglubokaja/
200 -- https://slova.org.ru/blok/brozhuvstenakh/
200 -- https://slova.org.ru/blok/budetden/
2

In [9]:
copy = poems

## 2) Деление на предложения
Добавляю в словарь уровень с предложениями.

In [10]:
def sentencize(text):
    sentences = nltk.sent_tokenize(text)
    result = {}
    for sent in sentences:
        sent = sent.replace("\n", " ")
        sent = ' '.join(tokenizer.tokenize(sent))
        result.update({sent: tokenizer.tokenize(sent)})
        
    return result

Оставляю только стихотворения, подходящие по регламенту (от 100 слов)

In [11]:
new = {}
for poem in poems:
    if len(poems[poem].split()) > 99:
        sents = sentencize(poems[poem])
        for sent in sents:
            new.update({poem: sentencize(poems[poem])})

## 3) Морфологический разбор
Добавляю в словарь уровень разбора. Для каждого слова мне нужна только лемма, словоформа и часть речи.

In [20]:
from pymystem3 import Mystem
m = Mystem(disambiguation=True)

In [21]:
morphed = {}
pos_list = set()
for poem in new:
    for sent in new[poem]:
        morphed_sent = []
        ana = m.analyze(sent)
        for word in ana:
            text = word['text']
            if text != " " and text != "\n":
                if "analysis" in word.keys() and word['analysis'] != []:
                    pos = word['analysis'][0]['gr'].split('=')[0].split(',')[0]
                    lex = word['analysis'][0]['lex']
                    pos_list.add(pos)
                else:
                    pos = None
                    lex = text
                anadict = {'text': text, 'lex': lex, 'pos': pos}
                morphed_sent.append(anadict)
        new[poem].update({sent: morphed_sent})

In [18]:
with open(PATH, 'wb') as f:
    pickle.dump(new, f)

In [5]:
with open(PATH, 'rb') as f:
    new = pickle.load(f)

In [8]:
before_df = []
for poem in new:
    for sent in new[poem]:
        for word in new[poem][sent]:
            before_df.append([poem, sent, word])

In [138]:
df = pd.DataFrame(before_df, columns = ["poem", "sentence", "word"])

In [10]:
df

,poem,sentence,word
0,Авиатор,Летун отпущен на свободу,"{'text': 'Летун', 'lex': 'летун', 'pos': 'S'}"
1,Авиатор,Летун отпущен на свободу,"{'text': 'отпущен', 'lex': 'отпускать', 'pos':..."
2,Авиатор,Летун отпущен на свободу,"{'text': 'на', 'lex': 'на', 'pos': 'PR'}"
3,Авиатор,Летун отпущен на свободу,"{'text': 'свободу', 'lex': 'свобода', 'pos': 'S'}"
4,Авиатор,Качнув две лопасти свои Как чудище морское в в...,"{'text': 'Качнув', 'lex': 'качнуть', 'pos': 'V'}"
...,...,...,...
32706,Religio,Со мной всю жизнь один Завет Завет служенья Не...,"{'text': 'один', 'lex': 'один', 'pos': 'APRO'}"
32707,Religio,Со мной всю жизнь один Завет Завет служенья Не...,"{'text': 'Завет', 'lex': 'завет', 'pos': 'S'}"
32708,Religio,Со мной всю жизнь один Завет Завет служенья Не...,"{'text': 'Завет', 'lex': 'завет', 'pos': 'S'}"
32709,Religio,Со мной всю жизнь один Завет Завет служенья Не...,"{'text': 'служенья', 'lex': 'служение', 'pos':..."


## 4) Подготавливаем функции для поиска

При поиске слова без кавычек я хочу учитывать омонимию, поэтому использую в этот раз пайморфи для получения нескольких разборов.

In [24]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [169]:
def is_match(req, word):
    res = False
    if req.startswith("\"") and req.endswith("\""):
        if word['text'].lower() == req.strip("\"").lower():
            res = True
    elif req.startswith("\"") == False and req.endswith("\"") == False:
        if req in pos_list:
            if word['pos'] == req:
                res = True
            
        elif req not in pos_list:
            if "+" in req:
                if word['text'].lower() == req.split("+")[0].lower() and word['pos'] == req.split("+")[1]:
                     #ищет слофоформу + часть речи
                    res = True
            else:
                for a in morph.parse(req):
                    lexeme = a.normal_form
                    if word['lex'] == lexeme:
                        res = True
    return res

In [157]:
def filter_search(req):
    filtered=pd.DataFrame(columns = ["poem", "sentence", "word"])
    for i in range(len(df) - len(req.split())):
        if is_match(req.split()[0], df.word[i]):
            
            if len(req.split()) > 1:
                if df.sentence[i] == df.sentence[i+1]:
                    if is_match(req.split()[1], df.word[i+1]):
                        if len(req.split()) > 2: 
                            if df.sentence[i] == df.sentence[i+2]:
                                if is_match(req.split()[2], df.word[i+2]):
                                    filtered = filtered.append(df[df['sentence'] == df.sentence[i]])
                                    #select.append(df.sentence[i])
                        else:
                            filtered = filtered.append(df[df['sentence'] == df.sentence[i]])
                            #select.append(df.sentence[i])
            else:
                filtered = filtered.append(df[df['sentence'] == df.sentence[i]])
                
    return filtered

In [158]:
def search(req):
    minidf = filter_search(req).drop(["word"], axis = 1).drop_duplicates()
    minidf.insert(loc=0, column='author', value=author)
    minidf.insert(loc=0, column='link', value=URL)
    return minidf

In [159]:
pd.options.display.max_colwidth = 3000

## 5) Поиск

In [183]:
search(input("Введите поисковой запрос: "))

Введите поисковой запрос: кулак


,link,author,poem,sentence
12993,https://slova.org.ru/blok/,Александр Блок,Забывшие Тебя,И руки повисали словно плети Когда вокруг сжимались кулаки Грозящие громам рыдали дети И жены кутались в платки


Слово в кавычках находит точное совпадение словоформы

In [184]:
search("пил") #пил

,link,author,poem,sentence
5204,https://slova.org.ru/blok/,Александр Блок,Возмездие,И ведь нимало не прельщает Армейский демонизм его Нет мой герой довольно тонок И прозорлив чтобы не знать Как бедный мучится ребенок Что счастие ребенку дать Теперь в его единой власти Нет нет но замерли в груди Доселе пламенные страсти И кто то шепчет погоди То ум холодный ум жестокий Вступил в нежданные права То муку жизни одинокой Предугадала голова Нет он не любит он играет Твердит она судьбу кляня За что терзает и пугает Он беззащитную меня Он объясненья не торопит Как будто сам чего то ждет Смотри так хищник силы копит Сейчас больным крылом взмахнет На луг опустится бесшумно И будет пить живую кровь Уже от ужаса безумной Дрожащей жертвы Вот любовь Того вампирственного века Который превратил в калек Достойных званья человека
7580,https://slova.org.ru/blok/,Александр Блок,Возмездие,Он жадно пьет с врагом вино И ветер ломится в окно Взывая к совести и к жизни Не также ль и тебя Варшава Столица гордых поляко в Дремать принудила орава Военных русских пошляков
8529,https://slova.org.ru/blok/,Александр Блок,Гимн,И по улицам словно бесчисленных пил Смех и скрежет и визг
8983,https://slova.org.ru/blok/,Александр Блок,"«Грешить бесстыдно, непробудно...»",И под лампадой у иконы Пить чай отщелкивая счет Потом переслюнить купоны Пузатый отворив комод И на перины пуховые В тяжелом завалиться сне Да и такой моя Россия Ты всех краев дороже мне
12457,https://slova.org.ru/blok/,Александр Блок,Жизнь моего приятеля,На утешенье на забаву Пей искрометное вино Пока вино тебе по нраву Пока не тягостно оно
15870,https://slova.org.ru/blok/,Александр Блок,«Лазурью бледной месяц плыл...»,У женщин взор был тускл и туп И страшен был их взор Я знал что судороги губ Открыли их позор Что пили ночь и забытьё Но день их опалил Как страшно мирное жильё Для тех кто изменил
21390,https://slova.org.ru/blok/,Александр Блок,Песнь Ада,И пил я кровь из плеч благоуханных И был напиток душен и смолист Но не кляни повествований странных О том как длился непонятный сон Из бездн ночных и пропастей туманных К нам доносился погребальный звон Язык огня взлетел свистя над нами Чтоб сжечь ненужность прерванных времен
23532,https://slova.org.ru/blok/,Александр Блок,Послания,ВАЛЕРИЮ БРЮСОВУ И вновь и вновь твой дух таинственный В глухой ночи в ночи пустой Велит к твоей мечте единственной Прильнуть и пить напиток твой
23557,https://slova.org.ru/blok/,Александр Блок,Послания,Вновь причастись души неистовой И яд и боль и сладость пей И тихо книгу перелистывай Впиваясь в зеркало теней Пусть несказанной мукой мучая Здесь бьется страсть змеится грусть Восторженная буря случая Сулит конец убийство пусть


Можно указать часть речи словоформы

Список частей речи: 'A',
 'ADV',
 'ADVPRO',
 'ANUM',
 'APRO',
 'CONJ',
 'INTJ',
 'NUM',
 'PART',
 'PR',
 'S',
 'SPRO',
 'V'

In [185]:
search("пил+S") #пил+S

,link,author,poem,sentence
8529,https://slova.org.ru/blok/,Александр Блок,Гимн,И по улицам словно бесчисленных пил Смех и скрежет и визг


In [186]:
search("пил+V") #пил+

,link,author,poem,sentence
21390,https://slova.org.ru/blok/,Александр Блок,Песнь Ада,И пил я кровь из плеч благоуханных И был напиток душен и смолист Но не кляни повествований странных О том как длился непонятный сон Из бездн ночных и пропастей туманных К нам доносился погребальный звон Язык огня взлетел свистя над нами Чтоб сжечь ненужность прерванных времен


Можно просто написать любую словоформу лексемы - поиск найдет вхождения всех словоформ

In [187]:
search("пил")

,link,author,poem,sentence
5204,https://slova.org.ru/blok/,Александр Блок,Возмездие,И ведь нимало не прельщает Армейский демонизм его Нет мой герой довольно тонок И прозорлив чтобы не знать Как бедный мучится ребенок Что счастие ребенку дать Теперь в его единой власти Нет нет но замерли в груди Доселе пламенные страсти И кто то шепчет погоди То ум холодный ум жестокий Вступил в нежданные права То муку жизни одинокой Предугадала голова Нет он не любит он играет Твердит она судьбу кляня За что терзает и пугает Он беззащитную меня Он объясненья не торопит Как будто сам чего то ждет Смотри так хищник силы копит Сейчас больным крылом взмахнет На луг опустится бесшумно И будет пить живую кровь Уже от ужаса безумной Дрожащей жертвы Вот любовь Того вампирственного века Который превратил в калек Достойных званья человека
7580,https://slova.org.ru/blok/,Александр Блок,Возмездие,Он жадно пьет с врагом вино И ветер ломится в окно Взывая к совести и к жизни Не также ль и тебя Варшава Столица гордых поляко в Дремать принудила орава Военных русских пошляков
8529,https://slova.org.ru/blok/,Александр Блок,Гимн,И по улицам словно бесчисленных пил Смех и скрежет и визг
8983,https://slova.org.ru/blok/,Александр Блок,"«Грешить бесстыдно, непробудно...»",И под лампадой у иконы Пить чай отщелкивая счет Потом переслюнить купоны Пузатый отворив комод И на перины пуховые В тяжелом завалиться сне Да и такой моя Россия Ты всех краев дороже мне
12457,https://slova.org.ru/blok/,Александр Блок,Жизнь моего приятеля,На утешенье на забаву Пей искрометное вино Пока вино тебе по нраву Пока не тягостно оно
15870,https://slova.org.ru/blok/,Александр Блок,«Лазурью бледной месяц плыл...»,У женщин взор был тускл и туп И страшен был их взор Я знал что судороги губ Открыли их позор Что пили ночь и забытьё Но день их опалил Как страшно мирное жильё Для тех кто изменил
21390,https://slova.org.ru/blok/,Александр Блок,Песнь Ада,И пил я кровь из плеч благоуханных И был напиток душен и смолист Но не кляни повествований странных О том как длился непонятный сон Из бездн ночных и пропастей туманных К нам доносился погребальный звон Язык огня взлетел свистя над нами Чтоб сжечь ненужность прерванных времен
23532,https://slova.org.ru/blok/,Александр Блок,Послания,ВАЛЕРИЮ БРЮСОВУ И вновь и вновь твой дух таинственный В глухой ночи в ночи пустой Велит к твоей мечте единственной Прильнуть и пить напиток твой
23557,https://slova.org.ru/blok/,Александр Блок,Послания,Вновь причастись души неистовой И яд и боль и сладость пей И тихо книгу перелистывай Впиваясь в зеркало теней Пусть несказанной мукой мучая Здесь бьется страсть змеится грусть Восторженная буря случая Сулит конец убийство пусть


Можно найти все вхождения определенной части речи

In [192]:
search('NUM')

,link,author,poem,sentence
4,https://slova.org.ru/blok/,Александр Блок,Авиатор,Качнув две лопасти свои Как чудище морское в воду Скользнул в воздушные струи
1633,https://slova.org.ru/blok/,Александр Блок,Возмездие,Сыны отражены в отцах Коротенький обрывок рода Два три звена и уж ясны Заветы темной старины Созрела новая порода Угль превращается в алмаз
2036,https://slova.org.ru/blok/,Александр Блок,Возмездие,Уж осень семьдесят восьмую Дотягивает старый век
2158,https://slova.org.ru/blok/,Александр Блок,Возмездие,За ними снежные Балканы Три Плевны Шипка и Дубняк Незаживающие раны И хитрый и неслабый враг Вон павловцы вон гренадеры По пыльной мостовой идут Их лица строги груди серы Блестит Георгий там и тут Разрежены их батальоны Но уцелевшие в бою Теперь под рваные знамена Склонили голову свою Конец тяжелого похода Незабываемые дни
3050,https://slova.org.ru/blok/,Александр Блок,Возмездие,И сонм собравшихся затих Два слова два рукопожатья И гость к ребенку в черном платье Идет минуя остальных Он смотрит долго и любовно И крепко руку жмет не раз И молвит Поздравляю вас С побегом Соня Софья Львовна
...,...,...,...,...
31450,https://slova.org.ru/blok/,Александр Блок,Черная кровь,Три свечи в бесконечной дали
31461,https://slova.org.ru/blok/,Александр Блок,Черная кровь,И под смуглым огнем трех свеч Смуглый бархат открытых плеч Буря спутанных кос тусклый глаз На кольце померкший алмаз И обугленный рот в крови Еще просит пыток любви А в провале глухих око н Смутный шелест многих знамен Звон и трубы и конский топ И качается тяжкий гроб
32202,https://slova.org.ru/blok/,Александр Блок,«Я жалобной рукой сжимаю свой костыль...»,И трое мы бредём
32283,https://slova.org.ru/blok/,Александр Блок,«Я живу в глубоком покое...»,Но в туманный вечер нас двое


### Примеры поиска
Можно вписать до трёх слов, каждое из которых в любом из форматов, описанных выше в любом сочетании.

In [193]:
search("INTJ S V")

,link,author,poem,sentence
9357,https://slova.org.ru/blok/,Александр Блок,Двенадцать,Ох большевики загонят в гроб
10340,https://slova.org.ru/blok/,Александр Блок,Двенадцать,Эй товарищ будет худо Выходи стрелять начнем


In [205]:
search("ADV я \"любил\"")

,link,author,poem,sentence
8500,https://slova.org.ru/blok/,Александр Блок,"«Всё это было, было, было...»",Но верю не пройдет бесследно Всё что так страстно я любил Весь трепет этой жизни бедной Весь этот непонятный пыл


In [200]:
search("A \"сну\"")

,link,author,poem,sentence
14583,https://slova.org.ru/blok/,Александр Блок,Из газет,Голубому сну еще рад наяву


In [202]:
search("A бред")

,link,author,poem,sentence
7984,https://slova.org.ru/blok/,Александр Блок,Возмездие,Лишь снег порхает вечный белый Зимой он площадь оснежит И мертвое засыплет тело Весной ручьями побежит Но в мыслях моего героя Уже почти несвязный бред Идет По снегу вьется след Один но их как было двое В ушах какой то смутный звон Вдруг бесконечная ограда Саксонского должно быть сада К ней тихо прислонился он
8710,https://slova.org.ru/blok/,Александр Блок,«Говорили короткие речи...»,Из отрывков слов я узнал Сумасшедший бред о невесте О том что кто то бежал
15631,https://slova.org.ru/blok/,Александр Блок,«Когда я стал дряхлеть и стынуть...»,Какой то образ прежде милый Мне снится в старческом бреду
31387,https://slova.org.ru/blok/,Александр Блок,Черная кровь,7 Ночь как века и томный трепет И страстный бред Уст о блаженно странном лепет В окне старинный слабый свет


In [204]:
search("PR \"мне\"")

,link,author,poem,sentence
10782,https://slova.org.ru/blok/,Александр Блок,Демон («Прижмись ко мне крепче и ближе...»),Прижмись ко мне крепче и ближе Не жил я блуждал средь чужих О сон мой
12136,https://slova.org.ru/blok/,Александр Блок,Жизнь моего приятеля,5 Пристал ко мне нищий дурак Идет по пятам как знакомый
21703,https://slova.org.ru/blok/,Александр Блок,Песня Фаины,Не подходи и ты ко мне Кого люблю и жду
27030,https://slova.org.ru/blok/,Александр Блок,СНЕГА,Но в гневах стали звенящей и падающей Твоя улыбка струится во мне
29076,https://slova.org.ru/blok/,Александр Блок,«Там дамы щеголяют модами...»,Там где скучаю так мучительно Ко мне приходит иногда Она бесстыдно упоительна И унизительно горда
30417,https://slova.org.ru/blok/,Александр Блок,«Ушла. Но гиацинты ждали...»,Вползи ко мне змеей ползучей В глухую полночь оглуши Устами томными замучай Косою черной задуши
30484,https://slova.org.ru/blok/,Александр Блок,"«Хожу, брожу понурый...»",Светла её одежда Она придёт ко мне
31035,https://slova.org.ru/blok/,Александр Блок,Черная кровь,1 В пол оборота ты встала ко мне Грудь и рука твоя видится мне
31115,https://slova.org.ru/blok/,Александр Блок,Черная кровь,Каждый демон во мне Притаился глядит
31552,https://slova.org.ru/blok/,Александр Блок,Черная кровь,Знаю выпил я кровь твою Я кладу тебя в гроб и пою Мглистой ночью о нежной весне Будет петь твоя кровь во мне


In [ ]:
search("")

In [ ]:
search("")